In [2]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/opt/conda/bin/python
3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) 
[GCC 7.5.0]
sys.version_info(major=3, minor=7, micro=8, releaselevel='final', serial=0)


In [3]:
from google.cloud import translate

In [15]:
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

#     print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}\n".format(result["translatedText"]))
#     print(u"Detected source language: {}\n".format(result["detectedSourceLanguage"]))


In [16]:
def get_pages(file_path):
    client = vision_v1.ImageAnnotatorClient()
    
    if isinstance(file_path, six.binary_type):
        file_path = file_path.decode('utf-8')
    
    mime_type = 'application/pdf'
    with io.open(file_path, 'rb') as f:
        content = f.read()
    input_config = {'mime_type': mime_type, 'content': content}
    type_ = enums.Feature.Type.DOCUMENT_TEXT_DETECTION
    features_element = {'type': type_}
    features = [features_element]
    
    # Load the pdf to the PdfFileReader object with default settings
    with open(file_path, "rb") as pdf_file:
        pdf_reader = PdfFileReader(pdf_file)
        print(f"The total number of pages in the pdf document is {pdf_reader.numPages}")
        
    n = pdf_reader.numPages
    val=list(range(1,n+1))
    p = [val[i:i + 5] for i in range(0, len(val), 5)]  
    print(p)
#     print(p[0])
#     print(p[1])
    # requests_element = {'input_config': input_config, 'features': features, 'pages': pages}
    return p

In [17]:
###OCR

In [18]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import enums
import io
import six
import os
from PyPDF2 import PdfFileReader

In [19]:
pdf_path = "ExecutiveOrder 2020-06.pdf"
P = get_pages(pdf_path)

The total number of pages in the pdf document is 1
[[1]]


In [20]:
def get_text(file_path):
    client = vision_v1.ImageAnnotatorClient()
    
    if isinstance(file_path, six.binary_type):
        file_path = file_path.decode('utf-8')
    
    mime_type = 'application/pdf'
    with io.open(file_path, 'rb') as f:
        content = f.read()
    input_config = {'mime_type': mime_type, 'content': content}
    type_ = enums.Feature.Type.DOCUMENT_TEXT_DETECTION
    features_element = {'type': type_}
    features = [features_element]

    TEXT = []
    for pages in P:
        requests_element = {'input_config': input_config, 'features': features, 'pages': pages}
        requests = [requests_element]

        response = client.batch_annotate_files(requests)

        for i, image_response in enumerate((response.responses[0].responses)):
            Text = (image_response.full_text_annotation.text)
            TEXT.append(Text)
            
    return TEXT

In [21]:
PDF_TEXT = get_text(pdf_path)

In [22]:
Target = {'Hindi': 'hi','Chinese':'zh', 'German':'de','Urdu':'ur'}
language = ['Chinese','', 'Hindi', 'Urdu', '', 'German']

In [24]:
language = ['Hindi']
for k in range(len(PDF_TEXT)):
    for i in language:
        if (i == ''):
            continue
    translate_text(Target[i], PDF_TEXT[k])

Translation: का राज्य । लॉजिस्टिक एक्ज़ीक्यूटिव डिपार्टमेंट 16 SPRINGHIDILD, ILLINOIS ht INDEX DEPARTMENT MAR 15 2020 मार्च ऑफ़ द स्टेट ऑफ़ द स्टेट ऑफ़ द 15 मार्च, 2020 एग्जीक्यूटिव ऑर्डर 2020 - 06 AM-CODIDSE to COVID-19 (COVID 19) को 1922 (EXCUTIVE) को पूरा करने का आदेश। 4) इलिनोइस के गवर्नर जेबी प्रित्जकर ने 9 मार्च, 2020 को कोरोनावायरस रोग 2019 के प्रकोप के जवाब में इलिनोइस राज्य में सभी काउंटी को 9 मार्च, 2020 को एक आपदा क्षेत्र के रूप में घोषित किया। 19%); और, 13 मार्च, 2020 को, कार्यकारी आदेश 5 (2020) ने 30 मार्च, 2020 तक शैक्षिक उद्देश्यों के लिए 12 वीं कक्षा के छात्रों के लिए इलिनोइस में प्री-किंडरगार्टन की सेवा देने वाले इलिनोइस के सभी सार्वजनिक और निजी स्कूलों को बंद करने का आदेश दिया; इसके अलावा, इलिनोइस राज्य के गवर्नर के रूप में मुझ में निहित शक्तियों द्वारा, और इलिनोइस आपातकालीन प्रबंधन एजेंसी अधिनियम, 20 ILCS 3305, मैं इसके द्वारा धारा 7 (1), 7 (8), और 7 (12) के अनुसरण में, मैं इसके द्वारा कार्यकारी आदेश 5 (2020) में निम्नानुसार संशोधन करें: खंड 1। 17 मार्च, 2020 से शुरू

In [ ]:
!gsutil -m cp -r "" "gs://fca_retrain_output"